In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision.datasets

import numpy as np

import os
import cv2

import pandas as pd
import numpy as np 
from numpy import vstack
import matplotlib.pyplot as plt

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, mean_squared_error

from math import sqrt

import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical 

from datetime import datetime


from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch import Tensor
from torch.nn import Linear
from torch.nn import ReLU
from torch.nn import Sigmoid
from torch.nn import Module
from torch.optim import SGD
from torch.nn import BCELoss
from torch.nn import MSELoss
from torch.nn.init import kaiming_uniform_
from torch.nn.init import xavier_uniform_
import torch

from matplotlib import pyplot

from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.models import load_model

from tensorflow.keras import preprocessing
from tensorflow.keras import backend as K
from tensorflow.keras import models

import tensorflow as tf
import numpy as np

from tensorflow.keras.preprocessing import image

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.utils import to_categorical 
from tensorflow.keras.models import Model

from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

# Desenvolvendo modelo CNN
* Modelo pytorch somente tensores
* Modelo pytorch com dataloader e dataset
* Conversão NHWC para NCHW
* Pytorch dataloader, dataset e transformer para data augmentation
* Usando modelo pronto para predição
* Transfer learning

# Dataset abelhas

In [ ]:
#!ls /home/silvio/git/datasets/bees/wingsEval

X = []
y = []


# Percorre os arquivos no diretório de imagens para treinamento
def create_test_data(path):
    for p in os.listdir(path):

        # categoria da imagem é definida pelo nome do arquivo
        category = p.split(" ")[0]
        
        # Abre a imagem usando opencv em escala de cinza
        #img_array = cv2.imread(os.path.join(path,p),cv2.IMREAD_GRAYSCALE)
        img_array = cv2.imread(os.path.join(path,p),cv2.COLOR_BGR2RGB)

        # Redimensionamento para 80 x 80 pixels
        new_img_array = cv2.resize(img_array, dsize=(320, 320))
        
        X.append(new_img_array)
        y.append(category)
        
path="/home/silvio/git/datasets/bees/wingsEval"

create_test_data(path)

# criando tensores de entrada

In [ ]:
le = LabelEncoder()
aux=le.fit_transform(y)

tX = Tensor(X)
ty = Tensor(aux)
#X=np.array(X)
#y=np.array(y)

print(tX.shape)
print(ty.shape)

# alterando de NHWC para NCHW

In [ ]:
tX = tX.permute(0, 3, 1, 2) 

# clase convnet

In [ ]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.drop_out = nn.Dropout()
        #self.fc1 = nn.Linear(7 * 7 * 64, 1000)
        #self.fc2 = nn.Linear(1000, 26)
        #100x409600
        self.fc1 = nn.Linear(409600, 100)
        self.fc2 = nn.Linear(100, 26)


    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.drop_out(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

# Rotina de treino (Usando tensores)

```
batche_size=100
n_batches=18
max_epochs=2

for epoch in range(max_epochs):
    for i in range(n_batches):
        local_X, local_y = tX[i*batche_size:(i+1)*batche_size,], ty[i*batche_size:(i+1)*batche_size,]
        #print(epoch,i,local_X.shape)
        #print(epoch,i,local_y.shape)
```

In [ ]:
batche_size=100
n_batches=18
max_epochs=20

learning_rate = 0.001

model = ConvNet()

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

loss_list = []
acc_list = []

# Train model
for epoch in range(max_epochs):
    for i in range(n_batches):
        local_X, local_y = tX[i*batche_size:(i+1)*batche_size,], ty[i*batche_size:(i+1)*batche_size,]

        outputs = model(local_X)
        
        local_y = local_y.to(torch.long)
        
        loss = criterion(outputs, local_y)
        loss_list.append(loss.item())

        # Backprop and perform Adam optimisation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Track the accuracy
        total = local_y.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == local_y).sum().item()
        acc_list.append(correct / total)

        print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
            .format(epoch + 1, max_epochs, i + 1, n_batches, loss.item(),
                (correct / total) * 100))

# Rotina de treino com dataset e dataloader

In [ ]:
num_epochs = 6
num_classes = 30
batch_size = 100
learning_rate = 0.001

* declarando dataset

In [ ]:
class wingsDataset(Dataset):
    
    def __init__(self, path):

        Xl = []
        yl = []
        
        for p in os.listdir(path):
            # categoria da imagem é definida pelo nome do arquivo
            category = p.split(" ")[0]
        
            # Abre a imagem usando opencv em escala de cinza
            #img_array = cv2.imread(os.path.join(path,p),cv2.IMREAD_GRAYSCALE)
            # Redimensionamento para 28 x 28 pixels
            #new_img_array = cv2.resize(img_array, dsize=(28, 28))

            img_array = cv2.imread(os.path.join(path,p),cv2.COLOR_BGR2RGB)

            # Redimensionamento para 80 x 80 pixels
            new_img_array = cv2.resize(img_array, dsize=(320, 320))            
            
            Xl.append(new_img_array)
            yl.append(category)
            #print(p)
        le = LabelEncoder()
        yll= le.fit_transform(yl)
        
        #yll=yll.astype(double)
        #print(yll)
        Xl=np.array(Xl)
        
        print(Xl.shape)
        Xl = np.array(Xl).reshape(-1, 320, 320,3)
        print(Xl.shape)
        
        self.X = Tensor(Xl)
        self.y = Tensor(np.array(yll))
        print(self.X.shape)
        print(torch.unique(self.y))
        self.X = self.X.permute(0, 3, 1, 2) # from NHWC to NCHW
         
    # quantas linhas tem no dataset?
    def __len__(self):
        return len(self.X)
 
    # obtem uma linha do dataset
    def __getitem__(self, idx):
        print(idx, self.X.shape)
        return [self.X[idx], self.y[idx]]
 
    # retorna base para treino e teste
    def get_splits(self, n_test=0.33):
        test_size = round(n_test * len(self.X))
        train_size = len(self.X) - test_size
        return random_split(self, [train_size, test_size])



* instanciando dataset e dataloader

In [ ]:
path = '/home/silvio/dataset/bees/wingsEval/'
#path = '/home/silvio/dataset/bees/wingsEvalT/'
# Carrega Dataset   
dataset = wingsDataset(path)
# realiza split
train, test = dataset.get_splits()
   
# monta data loaders
train_loader = DataLoader(train, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test, batch_size=batch_size, shuffle=False)



In [ ]:
len(dataset)

In [ ]:
!ls /home/silvio/dataset/bees/wingsEvalT/ | wc -l

* treino do modelo

In [ ]:
model = ConvNet()

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
total_step = len(train_loader)
#print(total_step)
loss_list = []
acc_list = []
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Run the forward pass
        #print(images.shape, labels.shape)
        labels = labels.to(torch.long)

        outputs = model(images)
        #print(outputs)
        #print(labels)
        #print(torch.unique(labels))
        loss = criterion(outputs, labels)
        loss_list.append(loss.item())

        # Backprop and perform Adam optimisation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Track the accuracy
        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        acc_list.append(correct / total)

        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                          (correct / total) * 100))



In [ ]:

pyplot.plot(loss_list, label='train')
#pyplot.plot(acc_list, label='test')
pyplot.legend()
pyplot.show()

In [ ]:
pyplot.plot(acc_list, label='test')
pyplot.legend()
pyplot.show()

# dataloader exemplo gato - cachorro - data augmentation

# Usando modelo para predição

In [ ]:
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications.vgg16 import decode_predictions
from tensorflow.keras.applications.vgg16 import VGG16


from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications import Xception # TensorFlow ONLY
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import VGG19
from tensorflow.keras.applications import imagenet_utils

In [ ]:
model = VGG16()
#model = VGG19()
#model = InceptionV3()
#model = Xception()
#model = ResNet50

In [ ]:
image = load_img('/home/silvio/mug.jpg', target_size=(224, 224))
image = load_img('/home/silvio/bee.jpg', target_size=(224, 224))
#image = load_img('/home/silvio/mug.jpg', target_size=(299, 299))

In [ ]:
!ls /home/silvio/mug.jpg

In [ ]:
# Converter pixels em um array
image = img_to_array(image)

# acertar formato do array para o modelo
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))

# preprocessamento da imagem
image = preprocess_input(image)

# calcular probabilidade considerando todos os labels
yhat = model.predict(image)

# Codificar as probabilidades retornadas nos "labels"
label = decode_predictions(yhat)

for (i, (imagenetID, label, prob)) in enumerate(label[0]):
    print("{}. {}: {:.2f}%".format(i + 1, label, prob * 100))




# obtém identificação da imagem (Label)
#label = label[0][0]

# Classificação
#print('%s (%.2f%%)' % (label[1], label[2]*100))

# Reuso de topologia

In [ ]:


model = ResNet50(weights='imagenet')

img_path = '/home/silvio/mug.jpg'
img_path = '/home/silvio/bee.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

preds = model.predict(x)

#print('Predict:', decode_predictions(preds, top=3)[0])


In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
import numpy as np

model = VGG16(weights='imagenet', include_top=False)

#img_path = '/home/silvio/mug.jpg'
img_path = '/home/silvio/bee.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

#features = model.predict(x)

In [ ]:
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.models import Model
import numpy as np

base_model = VGG19(weights='imagenet')
model = Model(inputs=base_model.input, outputs=base_model.get_layer('block4_pool').output)

#img_path = '/home/silvio/mug.jpg'
img_path = '/home/silvio/bee.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

#block4_pool_features = model.predict(x)

# Transfer learning a partir da VGG16

# Preparando dados para o transfer learning

In [ ]:
arrX=np.array(X)
arrX.shape
arrX=np.array(X)
print(arrX.shape)

print(aux.shape)

yc = to_categorical(aux) 
x_train, x_test, y_train, y_test = train_test_split(arrX, yc, test_size=0.2, random_state = 42)
print(yc.shape)

# acoplando nova camada densa ao final do modelo VGG16
* GlobalAveragePooling2D

In [ ]:
# modelo pre treinado
base_model = InceptionV3(weights='imagenet', include_top=False)

# forma de desenho de rede neural baseada em grafo (keras)
x = base_model.output
x = GlobalAveragePooling2D()(x)

x = Dense(1024, activation='relu')(x)

predictions = Dense(25, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])


history=model.fit(  x_train,   y_train,   epochs=5,   validation_split=0.3)

# heatmap da ativação por camadas

In [ ]:
from tensorflow.keras import applications

vgg_model = applications.VGG16(weights='imagenet', include_top=True)
vgg_model = applications.VGG16(weights='imagenet', include_top=False)

from tensorflow.keras.layers import Input
input_tensor = Input(shape=(160, 160, 3))
vgg_model = applications.VGG16(weights='imagenet',
                               include_top=False,
                               input_tensor=input_tensor)

vgg_model.summary()

In [ ]:
vgg_model = applications.VGG16(weights='imagenet',
                               include_top=False,
                               input_shape=(160, 160, 3))

layer_dict = dict([(layer.name, layer) for layer in vgg_model.layers])

x = layer_dict['block2_pool'].output

x = Conv2D(filters=64, kernel_size=(3, 3), activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(10, activation='softmax')(x)

custom_model = Model(input=vgg_model.input, output=x)

for layer in custom_model.layers[:7]:
    layer.trainable = False


custom_model.compile(loss='categorical_crossentropy',
                     optimizer='rmsprop',
                     metrics=['accuracy'])

In [ ]:
image_size = 224


model = tf.keras.applications.vgg16.VGG16()
img_path = '/home/silvio/bee.jpg'
im = image.load_img(img_path, target_size=(224, 224))


img_tensor = preprocessing.image.img_to_array(im)
img_tensor = np.expand_dims(img_tensor, axis=0)
img_tensor = preprocess_input(img_tensor)

In [ ]:
plt.imshow(im);

In [ ]:
conv_layer = model.get_layer("block5_conv3")
heatmap_model = models.Model([model.inputs], [conv_layer.output, model.output])


with tf.GradientTape() as gtape:
    conv_output, predictions = heatmap_model(img_tensor)
    loss = predictions[:, np.argmax(predictions[0])]
    grads = gtape.gradient(loss, conv_output)
    pooled_grads = K.mean(grads, axis=(0, 1, 2))

heatmap = tf.reduce_mean(tf.multiply(pooled_grads, conv_output), axis=-1)
heatmap = np.maximum(heatmap, 0)
max_heat = np.max(heatmap)
if max_heat == 0:
    max_heat = 1e-10
heatmap /= max_heat

print(heatmap.shape)

import matplotlib.pyplot as plt
hm=np.squeeze(heatmap)
print(hm.shape)

plt.imshow(hm)

In [ ]:
!ls /home/silvio/bee.jpg

# imagem original

In [ ]:
img = cv2.imread('/home/silvio/bee.jpg')

plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

# heatmap da ativação

In [ ]:
hm2=hm

hm2 = cv2.merge([hm2,hm2,hm2])
hm2 = cv2.resize(hm2, (img.shape[1], img.shape[0]))
hm2 = np.uint8(255 * hm2)
hm2 = cv2.applyColorMap(hm2, cv2.COLORMAP_JET)
superimposed_img = hm2 * 0.4 + img 


In [ ]:
plt.imshow(hm2)

# heatmap + original sobreposta

In [ ]:
plt.imshow(superimposed_img)